In [1]:
import sparknlp
spark = sparknlp.start() 
# sparknlp.start(gpu=True) >> for training on GPUfrom sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
import pandas as pd
print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

23/01/21 22:51:16 WARN Utils: Your hostname, Vader resolves to a loopback address: 127.0.1.1; using 192.168.1.94 instead (on interface enp5s0f0)
23/01/21 22:51:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/sjhjrok/anaconda3/envs/NLP/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/sjhjrok/.ivy2/cache
The jars for the packages stored in: /home/sjhjrok/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-60cc3934-5a74-459f-a4fb-f9e9fd8086d8;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;4.2.2 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#l

23/01/21 22:51:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Spark NLP version 4.2.2
Apache Spark version: 3.3.1


In [2]:
#! wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/news_category_train.csv

In [3]:
#! wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/news_category_test.csv

In [4]:
trainDataset = spark.read \
      .option("header", True) \
      .csv("text_train.csv")
trainDataset.show(10, truncate=50)

+--------------------------------+--------+
|                     description|category|
+--------------------------------+--------+
|           i want to make coffee|      DA|
|                      where am i|      LR|
|              what can i do here|      AR|
|                    prepare meal|      DA|
|                use refrigerator|      DA|
|                         use fan|      DA|
|                        use oven|      DA|
|                       use stove|      DA|
|     i would like to wash sheets|      DA|
|i would like to watch television|      DA|
+--------------------------------+--------+
only showing top 10 rows



In [5]:
from pyspark.sql.functions import col
trainDataset.groupBy("category") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------+-----+
|category|count|
+--------+-----+
|     NAV|   52|
|      DA|   21|
|       N|    6|
|       P|    6|
|      CC|    4|
|      OC|    4|
|      SD|    4|
|     REV|    2|
|      PV|    2|
|      LR|    2|
|     FFV|    2|
|       0|    1|
|      AR|    1|
+--------+-----+



In [6]:
testDataset = spark.read \
      .option("header", True) \
      .csv("text_test.csv")
testDataset.groupBy("category") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------+-----+
|category|count|
+--------+-----+
|     NAV|   52|
|      DA|   21|
|       N|    6|
|       P|    6|
|      CC|    4|
|      OC|    4|
|      SD|    4|
|     REV|    2|
|      PV|    2|
|      LR|    2|
|     FFV|    2|
|       0|    1|
|      AR|    1|
+--------+-----+



In [7]:
# actual content is inside description column
document = DocumentAssembler()\
    .setInputCol("description")\
    .setOutputCol("document")
    
# we can also use sentence detector here 
# if we want to train on and get predictions for each sentence# downloading pretrained embeddings
use = UniversalSentenceEncoder.pretrained()\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")
# the classes/labels/categories are in category column
classsifierdl = ClassifierDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")\
  .setLabelColumn("category")\
  .setMaxEpochs(5)\
  .setEnableOutputLogs(True)
use_clf_pipeline = Pipeline(
    stages = [
        document,
        use,
        classsifierdl
    ])

use_pipelineModel = use_clf_pipeline.fit(trainDataset)


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ]tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ / ]Download done! Loading the resource.
[ \ ]

2023-01-21 22:51:34.835511: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[ / ]

[OK!]


2023-01-21 22:51:42.888250: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/5f4084d85eda_classifier_dl17736311454010180057
2023-01-21 22:51:42.919150: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2023-01-21 22:51:42.919176: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: /tmp/5f4084d85eda_classifier_dl17736311454010180057
2023-01-21 22:51:43.172932: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2023-01-21 22:51:43.570142: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/5f4084d85eda_classifier_dl17736311454010180057
2023-01-21 22:51:43.669362: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. Took 781122 microsec

Training started - epochs: 5 - learning_rate: 0.005 - batch_size: 64 - training_examples: 107 - classes: 13
Epoch 1/5 - 0.19s - loss: 5.1370525 - acc: 0.44222385 - batches: 2
Epoch 2/5 - 0.01s - loss: 4.395865 - acc: 0.94222385 - batches: 2
Epoch 3/5 - 0.01s - loss: 4.363697 - acc: 0.94222385 - batches: 2
Epoch 4/5 - 0.01s - loss: 4.3477306 - acc: 0.94222385 - batches: 2
Epoch 5/5 - 0.01s - loss: 4.3275795 - acc: 0.94222385 - batches: 2


In [8]:
use_pipelineModel.stages[2].write().overwrite().save('Text_Classification')

In [9]:
# actual content is inside description column
document = DocumentAssembler()\
    .setInputCol("description")\
    .setOutputCol("document")
    
# we can also use sentence detector here 
# if we want to train on and get predictions for each sentence# downloading pretrained embeddings
use = UniversalSentenceEncoder.pretrained()\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")
# the classes/labels/categories are in category column
loaded_ner_model = ClassifierDLModel.load("Text_Classification")\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")

use_clf_pipeline = Pipeline(
    stages = [
        document,
        use,
        loaded_ner_model
    ])

#use_pipelineModel = use_clf_pipeline.fit(trainDataset)

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]


In [10]:
from sparknlp.base import LightPipeline
#clf_pipelineModel = use_clf_pipeline.fit(trainDataset)

text = "navigate to kitchen"
prediction_data = spark.createDataFrame([[text]]).toDF("text")
prediction_model = use_clf_pipeline.fit(prediction_data)
light = LightPipeline(prediction_model)
results = light.annotate(text)
print(results)
print(results['class'])

{'document': ['navigate to kitchen'], 'sentence_embeddings': ['navigate to kitchen'], 'class': ['NAV']}
['NAV']


In [11]:
text = input("Enter Testing Text\n")
while(text != "exit"):
    results = light.annotate(text)
    print(results['class'])
    text = input("Enter New Text\n")

Enter Testing Text
open camera
['NAV']
Enter New Text
close camera
['NAV']
Enter New Text
where am i
['NAV']
Enter New Text
make coffee
['DA']
Enter New Text
close camera 
['NAV']
Enter New Text
watch televesion 
['NAV']
Enter New Text
cook dinner
['DA']
Enter New Text
what can i do here
['NAV']
Enter New Text
exit
